In [1]:
import pandas as pd
import xml.etree.ElementTree as ET

In [2]:
def read_excell(file, sheet, row_end):
    # Read the Excel file
    df = pd.read_excel(file, sheet_name=sheet)

    # Limit the DataFrame to the specified number of rows
    df = df.iloc[:row_end]

    return df

def read_in_xml(path):
    tree = ET.parse(path)
    root = tree.getroot()

    # Define a function to extract data from XML elements
    def extract_data(element):
        return {
            child.tag: child.text
            for child in element
        }

    # Extract data from each element and convert it into a list of dictionaries
    data = [extract_data(child) for child in root]

    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)
    return df


In [3]:
df_display_table = read_in_xml('./data/Food_Display_Table.xml')
df_condiments_table = read_in_xml('data/Foods_Needing_Condiments_Table.xml')
df_lu_condiments = read_in_xml('data/lu_Condiment_Food_Table.xml')

In [11]:
df_condiments_table

,Survey_Food_Code,display_name,cond_1_code,cond_1_name,cond_2_code,cond_2_name,cond_3_code,cond_3_name,cond_4_code,cond_4_name,cond_5_code,cond_5_name
0,51208000,"100% Whole Wheat Bagel""",81101000,Butter,81102020,Tub margarine,81104010,Reduced calorie spread (margarine type),14301010,Cream cheese (regular),14303010,Low fat cream cheese
1,58100100,Beef burrito (no beans):,12310100,Sour cream,63409010,Guacamole,74402100,Salsa,NaN,NaN,NaN,NaN
2,58104740,Chicken & cheese quesadilla:,12310100,Sour cream,63409010,Guacamole,74402100,Salsa,NaN,NaN,NaN,NaN
3,23559,Ground beef (95% lean),74303000,Ketchup (catsup),74406010,Barbeque sauce,28500040,Beef gravy,74406100,Steak sauce (A-1),NaN,NaN
4,23564,Ground beef (90% lean),74303000,Ketchup (catsup),74406010,Barbeque sauce,28500040,Beef gravy,74406100,Steak sauce (A-1),NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
284,77300000,Wrap sandwich (vegetables only),12310100,Sour cream,63409010,Guacamole,74402100,Salsa,14104010,"Cheese (cheddar, Swiss, etc.)",14109030,"Lowfat cheese (cheddar, Swiss, etc.)"
285,92101000,Coffee,91101010,White sugar (granulated or lump),12120100,Half & Half,12210200,Coffee Mate creamer (liquid),12210250,Coffee Mate Lite creamer (liquid),11112210,1% milk (low fat)
286,92101900,Latte,91101010,White sugar (granulated or lump),12140000,"Whipped cream, sweetened",NaN,NaN,NaN,NaN,NaN,NaN
287,92161000,Cappuccino,91101010,White sugar (granulated or lump),12140000,"Whipped cream, sweetened",NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
df_lu_condiments_small = df_lu_condiments[["survey_food_code", "display_name"]].rename(columns={"survey_food_code": "foodCode"}).astype({"foodCode": int})
df_lu_condiments_small

,foodCode,display_name
0,11111000,Whole milk
1,11111000,Whole milk
2,11112210,1% milk (low fat)
3,11112210,1% milk (low fat)
4,11113000,Fat free milk (skim)
...,...,...
58,91300010,Syrup (regular)
59,91302010,Honey
60,91361010,Sweet and sour sauce
61,91401000,Jelly (all flavors)


In [62]:
# Check the category of each recipe
vegan_df = pd.read_json('./data/VegAttributes_for_FoodData_Central_survey_and_sr_legacy_food_json_2021-10-28-v1.json')
vegan_df['vegCategory'] = vegan_df['vegCategory'].str.lower()
vegan_df_small = vegan_df[~vegan_df["foodCode"].isna()]
vegan_df_small = vegan_df_small[["foodCode", "vegCategory"]].astype({"foodCode": int})
vegan_df_small

,foodCode,vegCategory
0,11000000,vegetarian
1,11100000,vegetarian
2,11111000,vegetarian
3,11111100,vegetarian
4,11111150,vegetarian
...,...,...
7078,99997802,vegan
7079,99997804,vegan
7080,99997810,vegan
7081,99998130,vegetarian


In [69]:
df_ingredients_vegan = pd.merge(df_lu_condiments_small, vegan_df_small, on = "foodCode")
df_ingredients_vegan

,foodCode,display_name,vegCategory
0,11111000,Whole milk,vegetarian
1,11111000,Whole milk,vegetarian
2,11112210,1% milk (low fat),vegetarian
3,11112210,1% milk (low fat),vegetarian
4,11113000,Fat free milk (skim),vegetarian
5,11113000,Fat free milk (skim),vegetarian
6,11320000,"Soymilk, plain (calcium fortified)",vegan
7,12110100,Light cream (coffee or table cream),vegetarian
8,12120100,Half & Half,vegetarian
9,12140000,Whipped cream (sweetened),vegetarian


In [27]:
# Recipes and Ingredients: Simple Cookbook
df_recipe_simple = pd.read_csv("./data/recipe.csv")
df_ingredient_simple = pd.read_csv("./data/ingredient.csv")

df_ingredient_simple.head()

,recipe_title,ingredient_name,ingredient_category,max_qty,min_qty,unit,preparation,optional
0,Raspberry Chiffon Pie,graham cracker crumbs,cookies/crackers,2.00,2.00,cup(s),NaN,False
1,Raspberry Chiffon Pie,sugar,baking products,0.25,0.25,cup(s),NaN,False
2,Raspberry Chiffon Pie,margarine or butter,dairy,0.50,0.50,cup(s),melted,False
3,Raspberry Chiffon Pie,raspberry juice,fruit juices,0.25,0.25,cup(s),or water,False
4,Raspberry Chiffon Pie,unflavored gelatin,gelatin,3.00,3.00,teaspoon(s),NaN,False


In [28]:
df_recipe_simple.head()

,recipe_title,sub_title,servings,yield_unit,prep_min,cook_min,stnd_min,source,intro,directions,...,cholestrl,sodium,iron,vitamin_c,vitamin_a,fiber,pcnt_cal_carb,pcnt_cal_fat,pcnt_cal_prot,calories
0,Raspberry Chiffon Pie,NaN,10,1 pie,20.0,8.0,305.0,The California Tree Fruit Agreement,NaN,"For crust, preheat oven to 375 degrees F.\nIn...",...,1.39,260.78,0.81,8.89,586.20,0.87,56.80,35.68,7.53,290.79
1,Apricot Yogurt Parfaits,NaN,4,NaN,5.0,2.0,65.0,Produce for Better Health Foundation and 5 a Day,NaN,"Drain canned apricots, pour 1/4 cup of the ju...",...,3.48,46.17,0.57,13.02,2738.24,0.62,67.38,6.89,16.17,141.01
2,Fresh Apricot Bavarian,NaN,8,NaN,5.0,13.0,NaN,The California Apricot Advisory Board,Serve in stemmed glasses and top with sliced a...,Drop apricots into boiling water to cover. R...,...,3.46,41.79,0.37,6.13,1521.10,0.34,78.45,7.24,14.30,137.06
3,Fresh Peaches,with Banana Cream Whip,4,NaN,10.0,NaN,NaN,Produce for Better Health Foundation and 5 a Day,"For a quick, low-cal dessert, serve this on o...","In a small bowl, beat egg white until foamy. ...",...,0.00,14.01,0.19,8.79,478.09,0.69,88.98,2.35,8.67,81.70
4,Canned Cherry Crisp,NaN,6,NaN,10.0,5.0,NaN,The Cherry Marketing Institute,Your microwave turns a can of cherry pie filli...,"Pour cherry pie filling into an 8-inch, round...",...,10.36,50.22,0.66,0.16,229.16,1.05,72.81,24.46,2.73,201.23


In [52]:
folder_path = "./data/atlas-query-cookbook"

df_ingredient = pd.read_csv(f"{folder_path}/Ingredient", sep='\t')
df_nutrition = pd.read_csv(f"{folder_path}/Nutrition", sep='\t')
df_quantity = pd.read_csv(f"{folder_path}/Quantity", sep='\t')
df_recipe = pd.read_csv(f"{folder_path}/Recipe", sep=',')

In [44]:
df_ingredient = (df_ingredient
                 .drop(columns = ["plural"])
                 .sort_values(by = "ingredient_id")
)
df_ingredient

,ingredient_id,category,name
0,1,dairy,1% lowfat cottage cheese
1,6,dairy,1% lowfat milk
2,10,Mexican products,10-inch flour tortilla
3,11,cereals,100% bran cereal
4,12,dairy,2% lowfat milk
...,...,...,...
3342,4641,cereals,cooked oatmeal
3343,4642,hot beverages,instant coffee granules
3344,4643,grains,long grain enriched white rice
3345,4644,frozen fruit,frozen grapefruit juice concentrate


In [35]:
df_nutrition

,recipe_id,protein,carbo,alcohol,total_fat,sat_fat,cholestrl,sodium,iron,vitamin_c,vitamin_a,fiber,pcnt_cal_carb,pcnt_cal_fat,pcnt_cal_prot,calories
0,214,5.47,41.29,0.00,11.53,2.21,1.39,260.78,0.81,8.89,586.20,0.87,56.80,35.68,7.53,290.79
1,215,5.70,23.75,1.93,1.08,0.58,3.48,46.17,0.57,13.02,2738.24,0.62,67.38,6.89,16.17,141.01
2,216,4.90,26.88,0.00,1.10,0.58,3.46,41.79,0.37,6.13,1521.10,0.34,78.45,7.24,14.30,137.06
3,217,1.77,18.17,0.00,0.21,0.06,0.00,14.01,0.19,8.79,478.09,0.69,88.98,2.35,8.67,81.70
4,218,1.38,36.63,0.00,5.47,3.46,10.36,50.22,0.66,0.16,229.16,1.05,72.81,24.46,2.73,201.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,1427,6.57,30.14,0.00,13.74,2.85,60.38,173.88,0.59,0.51,147.61,0.10,44.57,45.71,9.72,270.50
874,1428,9.13,29.91,0.00,16.88,9.12,144.36,458.80,1.80,0.39,644.66,0.10,38.83,49.31,11.86,308.10
875,1429,1.35,3.13,0.00,0.26,0.00,0.00,15.34,1.00,22.68,4048.92,0.20,61.69,11.73,26.59,20.31
876,1432,23.04,60.78,0.00,9.27,1.99,430.09,1075.80,9.06,109.82,12937.03,1.99,58.07,19.93,22.01,418.73


In [55]:
df_quantity

,quantity_id,recipe_id,ingredient_id,max_qty,min_qty,unit,preparation,optional
0,1,214,1613,2.00,2.00,cup(s),,False
1,2,214,3334,0.25,0.25,cup(s),,False
2,3,214,2222,0.50,0.50,cup(s),melted,False
3,4,214,2797,0.25,0.25,cup(s),or water,False
4,5,214,3567,3.00,3.00,teaspoon(s),,False
...,...,...,...,...,...,...,...,...
5087,6349,1119,289,6.00,6.00,pound(s),,False
5088,6350,1119,3261,750.00,750.00,milliliter(s),,False
5089,6351,1119,638,1.00,1.00,cup(s),,False
5090,6352,1119,1420,3.00,3.00,teaspoon(s),chopped,False


In [51]:
df_recipe

,recipe_id,title,subtitle,servings,yield_unit,prep_min,cook_min,stnd_min
0,214,Raspberry Chiffon Pie,NaN,10,1 pie,20,8,305
1,215,Apricot Yogurt Parfaits,NaN,4,NaN,5,2,65
2,216,Fresh Apricot Bavarian,NaN,8,NaN,5,13,0
3,217,Fresh Peaches,with Banana Cream Whip,4,NaN,10,0,0
4,218,Canned Cherry Crisp,NaN,6,NaN,10,5,0
...,...,...,...,...,...,...,...,...
873,1410,No-Bake Chocolate Cheesecake,NaN,12,NaN,30,0,120
874,1411,Mexican Cocoa Torte,NaN,10,NaN,90,12,120
875,1426,-Pancakes-,NaN,4,8,10,10,0
876,1427,-Waffles-,NaN,8,8 waffles,10,15,0


In [53]:
df_quantity_small = (df_quantity[["recipe_id", "ingredient_id"]]
                    .groupby("recipe_id")["ingredient_id"]
                    .agg(list)
                    .reset_index()
                    .rename(columns = {"ingredient_id": "ingredients"}))
df_quantity_small["ingredients"] = df_quantity_small["ingredients"].apply(lambda x: sorted(x))

df_recipe = df_recipe.drop(columns = ["source", "intro", "directions"])
df_recipe = pd.merge(df_recipe, df_quantity_small, on = "recipe_id")

df_recipe

,recipe_id,title,subtitle,servings,yield_unit,prep_min,cook_min,stnd_min,ingredients
0,214,Raspberry Chiffon Pie,NaN,10,1 pie,20,8,305,"[924, 1414, 1613, 2196, 2222, 2797, 3334, 3334..."
1,215,Apricot Yogurt Parfaits,NaN,4,NaN,5,2,65,"[71, 76, 1286, 2019, 2196, 3567]"
2,216,Fresh Apricot Bavarian,NaN,8,NaN,5,13,0,"[1286, 2196, 3334, 3567, 3649]"
3,217,Fresh Peaches,with Banana Cream Whip,4,NaN,10,0,0,"[792, 1402, 2019]"
4,218,Canned Cherry Crisp,NaN,6,NaN,10,5,0,"[30, 34, 351, 374, 614, 770, 1684, 2948]"
...,...,...,...,...,...,...,...,...,...
838,1410,No-Bake Chocolate Cheesecake,NaN,12,NaN,30,0,120,"[35, 249, 374, 374, 496, 782, 792, 921, 1755, ..."
839,1411,Mexican Cocoa Torte,NaN,10,NaN,90,12,120,"[738, 1755, 1767, 2627, 3118, 3329, 3334]"
840,1426,-Pancakes-,NaN,4,8,10,10,0,"[30, 126, 2327, 3021, 3334, 3595, 3618]"
841,1427,-Waffles-,NaN,8,8 waffles,10,15,0,"[30, 125, 1696, 2327, 3021, 3334, 3618]"
